# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
from src.match_names import name_fbref_to_fpl, neutralize_name

# Getting fbref

In [3]:
fbref = pd.read_csv("data/logs/logs_all_PL.csv")

In [4]:
fbref.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Date', 'Day', 'Comp',
       'Round', 'Venue', 'Squad', 'Opponent', 'Start', 'Pos', 'Min', 'Gls',
       'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Tkl',
       'Int', 'Blocks', 'xG', 'npxG', 'xAG', 'SCA', 'GCA', 'Cmp', 'Att',
       'Cmp%', 'PrgP', 'Carries', 'PrgC', 'Att.1', 'Succ', 'Match Report',
       'Name', 'Season', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'OG', 'PKwon',
       'PKcon', 'WDL', 'GoalsTeams', 'Team Score', 'Opp Score', 'CS',
       'Was Home', 'GW'],
      dtype='object')

In [5]:
fbref.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Date,Day,Comp,Round,...,OG,PKwon,PKcon,WDL,GoalsTeams,Team Score,Opp Score,CS,Was Home,GW
0,0,0.0,0.0,0.0,0.0,0.0,2021-08-21,Sat,Premier League,Matchweek 2,...,NaN,NaN,NaN,W,2–0,2,0,1.0,1.0,2
1,1,1.0,1.0,1.0,1.0,1.0,2021-08-28,Sat,Premier League,Matchweek 3,...,NaN,NaN,NaN,L,0–2,0,2,0.0,1.0,3
2,4,4.0,4.0,4.0,4.0,4.0,2021-09-19,Sun,Premier League,Matchweek 5,...,NaN,NaN,NaN,W,2–1,2,1,0.0,1.0,5
3,6,6.0,6.0,6.0,6.0,6.0,2021-09-27,Mon,Premier League,Matchweek 6,...,NaN,NaN,NaN,D,1–1,1,1,0.0,0.0,6
4,7,7.0,7.0,7.0,7.0,7.0,2021-10-02,Sat,Premier League,Matchweek 7,...,NaN,NaN,NaN,D,0–0,0,0,1.0,1.0,7


In [6]:
fbref.shape

(20243, 59)

In [7]:
fbref.isna().sum().sort_values().head(55)

Unnamed: 0                  0
Sh                          0
SoT                         0
CrdY                        0
CrdR                        0
Int                         0
Was Home                    0
PKatt                       0
Match Report                0
Season                      0
WDL                         0
GoalsTeams                  0
Team Score                  0
Opp Score                   0
CS                          0
Name                        0
PK                          0
GW                          0
Gls                         0
Start                       0
Opponent                    0
Squad                       0
Date                        0
Day                         0
Venue                       0
Comp                        0
Round                       0
Ast                         0
Min                         4
Pos                         4
Succ                      406
Att.1                     406
Carries                   406
PrgP      

In [8]:
# info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating"]
# fbref[fbref["Name"] == "Erling-Haaland"][["Name", "neutral_name", "GW", 'FPL_name', 'Season', 'FPL_team', 'Was Home', 'FPL_opp_team']]

# Getting FPL

In [9]:
fpl = pd.read_csv("data/previous/fpl_players.csv")

In [10]:
# fpl[(fpl["name"]=="Brenden Aaronson") & (fpl["season"]=="2022-23")]

In [11]:
fpl = fpl.drop(columns=["xP"])

In [12]:
fpl["date"] = fpl["kickoff_time"].str.split("T").str[0]

In [13]:
fpl.isna().sum().sort_values().head(55)

Unnamed: 0                        0
penalties_saved                   0
red_cards                         0
round                             0
saves                             0
selected                          0
team_a_score                      0
team_h_score                      0
own_goals                         0
threat                            0
transfers_balance                 0
transfers_in                      0
transfers_out                     0
value                             0
was_home                          0
yellow_cards                      0
GW                                0
total_points                      0
opp_team_name                     0
penalties_missed                  0
minutes                           0
season                            0
name                              0
position                          0
team                              0
assists                           0
bonus                             0
opponent_team               

In [14]:
# removing rows with GW == NaN
# fpl = fpl[fpl['GW'].notna()]

In [15]:
# fpl[(fpl["season"] == "2022-23") & (fpl["name"] == "Ben Davies") & (fpl["GW"] == 2)]

In [16]:
fbref["Season"].unique()

array(['2021-22', '2022-23', '2023-24'], dtype=object)

In [17]:
fpl.columns

Index(['Unnamed: 0', 'season', 'name', 'position', 'team', 'assists', 'bonus',
       'bps', 'clean_sheets', 'creativity', 'element', 'fixture',
       'goals_conceded', 'goals_scored', 'ict_index', 'influence',
       'kickoff_time', 'minutes', 'opponent_team', 'opp_team_name',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'yellow_cards', 'GW', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'starts', 'date'],
      dtype='object')

In [18]:
fpl.head()

,Unnamed: 0,season,name,position,team,assists,bonus,bps,clean_sheets,creativity,...,value,was_home,yellow_cards,GW,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts,date
0,0,2020-21,Aaron Connolly,FWD,Brighton,0,0,-3,0,0.3,...,55,True,0,1,NaN,NaN,NaN,NaN,NaN,2020-09-14
1,1,2020-21,Aaron Cresswell,DEF,West Ham,0,0,11,0,11.2,...,50,True,0,1,NaN,NaN,NaN,NaN,NaN,2020-09-12
2,2,2020-21,Aaron Mooy,MID,Brighton,0,0,0,0,0.0,...,50,True,0,1,NaN,NaN,NaN,NaN,NaN,2020-09-14
3,3,2020-21,Aaron Ramsdale,GK,Sheffield Utd,0,0,12,0,0.0,...,50,True,0,1,NaN,NaN,NaN,NaN,NaN,2020-09-14
4,4,2020-21,Abdoulaye Doucouré,MID,Everton,0,0,20,1,44.6,...,55,False,0,1,NaN,NaN,NaN,NaN,NaN,2020-09-13


In [19]:
fpl.shape

(78255, 44)

In [20]:
fpl.columns

Index(['Unnamed: 0', 'season', 'name', 'position', 'team', 'assists', 'bonus',
       'bps', 'clean_sheets', 'creativity', 'element', 'fixture',
       'goals_conceded', 'goals_scored', 'ict_index', 'influence',
       'kickoff_time', 'minutes', 'opponent_team', 'opp_team_name',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'yellow_cards', 'GW', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'starts', 'date'],
      dtype='object')

In [21]:
# info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating"]
# fpl[fpl["name"] == "Erling Haaland"][["name", "neutral_name", "GW", 'neutral_name', 'season', 'team', 'was_home', 'opp_team_name']]

In [22]:
# sorted(fpl["team"].unique())

In [23]:
# sorted(fbref["Squad"].unique())

# Merging

In [24]:
def team_fbref_to_fpl(team_name):
    if team_name == "Nott'ham Forest":
        return "Nott'm Forest"
    if team_name == "Leicester City":
        return "Leicester"
    if team_name == "Leeds United":
        return "Leeds"
    if team_name == "Manchester City":
        return "Man City"
    if team_name == "Manchester Utd":
        return "Man Utd"
    if team_name == "Tottenham":
        return "Spurs"
    if team_name == "Newcastle Utd":
        return "Newcastle"
    
    return team_name

In [25]:
# add neutral name columns
fpl["neutral_name"] = fpl["name"].apply(neutralize_name)
fbref["neutral_name"] = fbref["Name"].apply(neutralize_name)
fbref["FPL_name"] = fbref["neutral_name"].apply(name_fbref_to_fpl)
fbref["FPL_team"] = fbref["Squad"].apply(team_fbref_to_fpl)
fbref["FPL_opp_team"] = fbref["Opponent"].apply(team_fbref_to_fpl)

In [26]:
# fpl.info()

In [27]:
# fbref.info()

In [28]:
fpl["opp_team_name"].isna().sum()

0

In [29]:
# join_columns_fbref = ['FPL_name', 'Season', 'FPL_team', 'Was Home', 'FPL_opp_team']
# join_columns_fpl = ['neutral_name', 'season', 'team', 'was_home', 'opp_team_name']
join_columns_fbref = ['FPL_name', 'Date']
join_columns_fpl = ['neutral_name', 'date']

In [30]:
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating"]
fpl[fpl["name"] == "Erling Haaland"][["name", "neutral_name", "GW", 'neutral_name', 'season', 'team', 'was_home', 'opp_team_name', "minutes", "kickoff_time"]]

,name,neutral_name,GW,neutral_name,season,team,was_home,opp_team_name,minutes,kickoff_time
49886,Erling Haaland,Erling-Haaland,1,Erling-Haaland,2022-23,Man City,False,West Ham,77,2022-08-07T15:30:00Z
50460,Erling Haaland,Erling-Haaland,2,Erling-Haaland,2022-23,Man City,True,Bournemouth,73,2022-08-13T14:00:00Z
51042,Erling Haaland,Erling-Haaland,3,Erling-Haaland,2022-23,Man City,False,Newcastle,90,2022-08-21T15:30:00Z
51635,Erling Haaland,Erling-Haaland,4,Erling-Haaland,2022-23,Man City,True,Crystal Palace,83,2022-08-27T14:00:00Z
52237,Erling Haaland,Erling-Haaland,5,Erling-Haaland,2022-23,Man City,True,Nott'm Forest,68,2022-08-31T18:30:00Z
52845,Erling Haaland,Erling-Haaland,6,Erling-Haaland,2022-23,Man City,False,Aston Villa,90,2022-09-03T16:30:00Z
53442,Erling Haaland,Erling-Haaland,8,Erling-Haaland,2022-23,Man City,False,Wolves,90,2022-09-17T11:30:00Z
53911,Erling Haaland,Erling-Haaland,9,Erling-Haaland,2022-23,Man City,True,Man Utd,90,2022-10-02T13:00:00Z
54550,Erling Haaland,Erling-Haaland,10,Erling-Haaland,2022-23,Man City,True,Southampton,90,2022-10-08T14:00:00Z
55191,Erling Haaland,Erling-Haaland,11,Erling-Haaland,2022-23,Man City,False,Leeds,90,2022-10-16T15:30:00Z


In [31]:
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating"]
fbref[fbref["Name"] == "Erling-Haaland"][["Name", "neutral_name", "GW", 'FPL_name', 'Season', 'FPL_team', 'Was Home', 'FPL_opp_team', "Min", "Date"]]

,Name,neutral_name,GW,FPL_name,Season,FPL_team,Was Home,FPL_opp_team,Min,Date
11457,Erling-Haaland,Erling-Haaland,1,Erling-Haaland,2022-23,Man City,0.0,West Ham,77.0,2022-08-07
11458,Erling-Haaland,Erling-Haaland,2,Erling-Haaland,2022-23,Man City,1.0,Bournemouth,73.0,2022-08-13
11459,Erling-Haaland,Erling-Haaland,3,Erling-Haaland,2022-23,Man City,0.0,Newcastle,90.0,2022-08-21
11460,Erling-Haaland,Erling-Haaland,4,Erling-Haaland,2022-23,Man City,1.0,Crystal Palace,83.0,2022-08-27
11461,Erling-Haaland,Erling-Haaland,5,Erling-Haaland,2022-23,Man City,1.0,Nott'm Forest,68.0,2022-08-31
11462,Erling-Haaland,Erling-Haaland,6,Erling-Haaland,2022-23,Man City,0.0,Aston Villa,90.0,2022-09-03
11463,Erling-Haaland,Erling-Haaland,8,Erling-Haaland,2022-23,Man City,0.0,Wolves,90.0,2022-09-17
11464,Erling-Haaland,Erling-Haaland,9,Erling-Haaland,2022-23,Man City,1.0,Man Utd,90.0,2022-10-02
11465,Erling-Haaland,Erling-Haaland,10,Erling-Haaland,2022-23,Man City,1.0,Southampton,90.0,2022-10-08
11466,Erling-Haaland,Erling-Haaland,11,Erling-Haaland,2022-23,Man City,0.0,Liverpool,90.0,2022-10-16


In [32]:
result = fbref.merge(fpl, left_on=join_columns_fbref, right_on=join_columns_fpl, how='inner')

In [33]:
result = result.drop(columns=["date"])

In [34]:
result.columns

Index(['Unnamed: 0_x', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Date', 'Day', 'Comp',
       'Round',
       ...
       'value', 'was_home', 'yellow_cards', 'GW_y', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'starts', 'neutral_name_y'],
      dtype='object', length=107)

In [35]:
# result[result["Name"] == "Erling-Haaland"][["Name", "neutral_name", "GW", 'FPL_name', 'Season', 'FPL_team', 'Was Home', 'FPL_opp_team', "Min"]].shape

In [36]:
# result.columns.to_list()

In [37]:
result = result.drop(columns=["GW_y", "neutral_name_y"])
result = result.rename(columns={"GW_x": "GW", "neutral_name_x": "neutral_name"})
# result = result.drop(columns=["GW_y"])
# result = result.rename(columns={"GW_x": "GW", "neutral_name_x": "neutral_name"})

In [38]:
result.isna().sum().sort_values().tail(55)

SoT                               0
Sh                                0
PKatt                             0
PK                                0
Ast                               0
Gls                               0
Day                               0
Match Report                      0
Name                              0
Season                            0
Int                               0
Start                             0
Opponent                          0
Squad                             0
Venue                             0
Round                             0
Comp                              0
CrdY                              0
CrdR                              0
Min                               4
Pos                               4
Tkl                             350
Blocks                          350
Touches                         350
xAG                             350
Succ                            350
xG                              350
PrgC                        

In [39]:
result[(result["total_points"].isna())][["Name", "neutral_name", "GW", 'FPL_name', 'Season', 'FPL_team', 'Was Home', 'FPL_opp_team', "Min", "Date"]].tail(50)

,Name,neutral_name,GW,FPL_name,Season,FPL_team,Was Home,FPL_opp_team,Min,Date


In [40]:
result.shape

(18835, 105)

In [41]:
np.unique(result.columns)

array(['Ast', 'Att', 'Att.1', 'Blocks', 'CS', 'Carries', 'Cmp', 'Cmp%',
       'Comp', 'CrdR', 'CrdY', 'Crs', 'Date', 'Day', 'FPL_name',
       'FPL_opp_team', 'FPL_team', 'Fld', 'Fls', 'GCA', 'GW', 'Gls',
       'GoalsTeams', 'Int', 'Match Report', 'Min', 'Name', 'OG', 'Off',
       'Opp Score', 'Opponent', 'PK', 'PKatt', 'PKcon', 'PKwon', 'Pos',
       'PrgC', 'PrgP', 'Round', 'SCA', 'Season', 'Sh', 'SoT', 'Squad',
       'Start', 'Succ', 'Team Score', 'Tkl', 'TklW', 'Touches',
       'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0_x',
       'Unnamed: 0_y', 'Venue', 'WDL', 'Was Home', 'assists', 'bonus',
       'bps', 'clean_sheets', 'creativity', 'element', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time',
       'minutes', 'name', 'neutral_name', 'npxG', 'opp_t

# Saving merged df to file

In [42]:
# fbref[["Name", "GW", "Season", "Squad", "Total Points"]].to_csv("data/previous/fpl_fbref_players.csv")
result.to_csv("data/previous/fpl_fbref_players.csv")

In [43]:
result["name"].isna().sum()

0

In [44]:
result.columns[50:120]

Index(['PKwon', 'PKcon', 'WDL', 'GoalsTeams', 'Team Score', 'Opp Score', 'CS',
       'Was Home', 'GW', 'neutral_name', 'FPL_name', 'FPL_team',
       'FPL_opp_team', 'Unnamed: 0_y', 'season', 'name', 'position', 'team',
       'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element',
       'fixture', 'goals_conceded', 'goals_scored', 'ict_index', 'influence',
       'kickoff_time', 'minutes', 'opponent_team', 'opp_team_name',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'yellow_cards', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'starts'],
      dtype='object')

In [45]:
result[result["Name"] == "Erling-Haaland"][["Name", "name", "GW"]]

,Name,name,GW
10334,Erling-Haaland,Erling Haaland,1
10335,Erling-Haaland,Erling Haaland,2
10336,Erling-Haaland,Erling Haaland,3
10337,Erling-Haaland,Erling Haaland,4
10338,Erling-Haaland,Erling Haaland,5
10339,Erling-Haaland,Erling Haaland,6
10340,Erling-Haaland,Erling Haaland,8
10341,Erling-Haaland,Erling Haaland,9
10342,Erling-Haaland,Erling Haaland,10
10343,Erling-Haaland,Erling Haaland,11


In [46]:
result.columns

Index(['Unnamed: 0_x', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Date', 'Day', 'Comp',
       'Round',
       ...
       'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
       'expected_assists', 'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'starts'],
      dtype='object', length=105)